In [24]:
pip install boto3 botocore s3transfer

  Using cached boto3-1.34.145-py3-none-any.whl.metadata (6.6 kB)
  Using cached s3transfer-0.10.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached botocore-1.34.145-py3-none-any.whl.metadata (5.7 kB)
Using cached boto3-1.34.145-py3-none-any.whl (139 kB)
Using cached botocore-1.34.145-py3-none-any.whl (12.4 MB)
Using cached s3transfer-0.10.2-py3-none-any.whl (82 kB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.29
    Uninstalling botocore-1.34.29:
      Successfully uninstalled botocore-1.34.29
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awsebcli 3.20.10 requires botocore<1.32.0,>1.23.41, but you have botocore 1.34.145 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import boto3
import pandas as pd
import io
import re

# AWS S3 버킷 정보
bucket_name = 'papalio-test-bucket'
file_key = 'test_otto/products_with_size_color.csv'

# S3 클라이언트 생성
s3 = boto3.client('s3')

# S3에서 CSV 파일 다운로드
try:
    response = s3.get_object(Bucket=bucket_name, Key=file_key)
    content = response['Body'].read().decode('utf-8')
    
    # CSV 내용을 데이터프레임으로 변환
    df = pd.read_csv(io.StringIO(content))
    
    # 데이터프레임 출력
    #print(df)
    #df csv파일로 만들기
    
except Exception as e:
    print(f"Error reading S3 object: {e}")



In [2]:
import boto3
import pandas as pd
import io
import re
import json

'''
이 단은 product table을 전처리하는 부분임
'''

#production preprocessing
df['product_id'] = df.index.to_series().apply(lambda x: f'top_{x}')
# price 열의 데이터를 전처리
for i in range(len(df)):
    price_value = df.loc[i, 'price']
    # 빈칸을 기준으로 앞부분만 남기고 나머지 제거
    price_value = price_value.split(' ')[0]
    # 숫자와 쉼표만 남기기
    price_value = re.sub(r'[^\d,]', '', price_value)
    df.loc[i, 'price'] = price_value


#color_options 비우기

#category를 전부 top으로 바꾸기




In [4]:
df.to_csv("preprocessing_test.csv", index=False, encoding='utf-8-sig')

In [6]:
import pandas as pd
import json

# CSV 파일 읽기
df = pd.read_csv('preprocessing_test.csv')

# reviews 컬럼의 JSON 데이터를 파싱하는 함수
def parse_reviews(review_str):
    try:
        return json.loads(review_str.replace("'", '"'))
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return []

# 모든 리뷰를 담을 리스트
all_reviews = []

# 각 product_name별로 리뷰를 추출하여 통합
for index, row in df.iterrows():
    product_name = row['product_name']
    reviews_list = parse_reviews(row['reviews'])
    
    for review in reviews_list:
        review['product_name'] = product_name  # 각 리뷰에 제품 이름 추가
        all_reviews.append(review)
        
# 통합 리뷰 데이터를 데이터프레임으로 변환
reviews_df = pd.DataFrame(all_reviews)

# weight, height, gender를 분리하여 새로운 컬럼 생성
def split_weight_height_gender(whg):
    parts = whg.split(' · ')
    if len(parts) == 3:
        gender, height, weight = parts
        height = height.replace('cm', '')
        weight = weight.replace('kg', '')
        return gender.strip(), height.strip(), weight.strip()
    return None, None, None

reviews_df['gender'], reviews_df['height'], reviews_df['weight'] = zip(*reviews_df['weight_height_gender'].map(split_weight_height_gender))

# 컬럼명 변경
reviews_df.rename(columns={'top_size': 'size_option', 'purchased_size': 'size'}, inplace=True)

# 새로운 컬럼 top_size와 bottom_size 생성
reviews_df['top_size'] = 'none'
reviews_df['bottom_size'] = 'none'
# 컬럼 삭제
reviews_df.drop(columns=['weight_height_gender'], inplace=True)

# 통합 리뷰 데이터를 CSV 파일로 저장
reviews_df.to_csv('musinsa_reviews.csv', index=False, encoding='utf-8-sig')
print("Saved all reviews to musinsa_reviews.csv")


Error decoding JSON: Expecting ',' delimiter: line 1 column 549 (char 548)
Error decoding JSON: Expecting ',' delimiter: line 1 column 1827 (char 1826)
Error decoding JSON: Expecting ',' delimiter: line 1 column 436 (char 435)
Error decoding JSON: Invalid \escape: line 1 column 1440 (char 1439)
Saved all reviews to musinsa_reviews.csv


In [6]:
'''
이 코드는 내 전처리 파트임.
'''

import boto3
import pandas as pd
import io
import re
import json

# AWS S3 버킷 정보
bucket_name = 'papalio-test-bucket'
file_key = 'test_otto/products_with_size_color.csv'

# S3 클라이언트 생성
s3 = boto3.client('s3')

# S3에서 CSV 파일 다운로드
try:
    response = s3.get_object(Bucket=bucket_name, Key=file_key)
    content = response['Body'].read().decode('utf-8')
    
    # CSV 내용을 데이터프레임으로 변환
    df = pd.read_csv(io.StringIO(content))
    
    # 데이터프레임 출력
    #print(df)
    #df csv파일로 만들기
    
except Exception as e:
    print(f"Error reading S3 object: {e}")

# Production preprocessing
df['product_id'] = 'none'
df.insert(0, 'rank', [i for i in range(0, len(df))])

# price 열의 데이터를 전처리
for i in range(len(df)):
    price_value = df.loc[i, 'price']
    # 빈칸을 기준으로 앞부분만 남기고 나머지 제거
    price_value = price_value.split(' ')[0]
    # 숫자와 쉼표만 남기기
    price_value = re.sub(r'[^\d,]', '', price_value)
    df.loc[i, 'price'] = price_value

# color_options 비우기
df['color_options'] = 'none'

# category를 전부 'top'으로 바꾸기
df['category'] = 'top'
df.drop(columns=['size_options'],inplace=True)


# 리뷰 데이터 전처리
# reviews 컬럼의 JSON 데이터를 파싱하는 함수
def parse_reviews(review_str):
    try:
        return json.loads(review_str.replace("'", '"'))
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return []

# 모든 리뷰를 담을 리스트
all_reviews = []

# 각 product_name별로 리뷰를 추출하여 통합
for index, row in df.iterrows():
    product_name = row['product_name']
    reviews_list = parse_reviews(row['reviews'])
    
    for review in reviews_list:
        review['product_name'] = product_name  # 각 리뷰에 제품 이름 추가
        all_reviews.append(review)
        
# 통합 리뷰 데이터를 데이터프레임으로 변환
reviews_df = pd.DataFrame(all_reviews)

# weight, height, gender를 분리하여 새로운 컬럼 생성
def split_weight_height_gender(whg):
    parts = whg.split(' · ')
    if len(parts) == 3:
        gender, height, weight = parts
        height = height.replace('cm', '')
        weight = weight.replace('kg', '')
        return gender.strip(), height.strip(), weight.strip()
    return None, None, None

reviews_df['gender'], reviews_df['height'], reviews_df['weight'] = zip(*reviews_df['weight_height_gender'].map(split_weight_height_gender))

# weight_height_gender 컬럼 삭제
reviews_df.drop(columns=['weight_height_gender'], inplace=True)

# 컬럼명 변경
reviews_df.rename(columns={'top_size': 'size_option', 'purchased_size': 'size'}, inplace=True)
reviews_df.drop(columns=['purchased_product_id'],inplace=True)

# 새로운 컬럼 top_size와 bottom_size 생성
reviews_df['top_size'] = 'none'
reviews_df['bottom_size'] = 'none'
df.drop(columns=['reviews'],inplace=True)
# 통합 리뷰 데이터를 CSV 파일로 저장
reviews_df.to_csv('musinsa_reviews.csv', index=False, encoding='utf-8-sig')
print("Saved all reviews to musinsa_reviews.csv")

# 변경된 product 테이블도 CSV 파일로 저장
df.to_csv('processed_products.csv', index=False, encoding='utf-8-sig')
print("Saved processed products to processed_products.csv")


Error decoding JSON: Expecting ',' delimiter: line 1 column 549 (char 548)
Error decoding JSON: Expecting ',' delimiter: line 1 column 1827 (char 1826)
Error decoding JSON: Expecting ',' delimiter: line 1 column 436 (char 435)
Error decoding JSON: Invalid \escape: line 1 column 1440 (char 1439)
Saved all reviews to musinsa_reviews.csv
Saved processed products to processed_products.csv
